In [1]:
import pickle

import matplotlib.pyplot as plt
import promptlib
import seaborn as sns

from face_recognition.api import get_known_people_from_encodings, get_known_people_from_dataset
from face_recognition.web_prediction import predict
from test.video_prediction import produce_video

method = input('Test by:\n (1) Webcam\n(2) video')
output_faces = []
person_name = ''

if method != '1' and method != '2':
    exit(1)
elif method == '1':
    name = input("Enter your name: ")
    faces = predict()
    names_in_db = get_known_people_from_dataset()
    x = {}
    print(faces)
    for nume in names_in_db:
        x[nume] = (faces.count(nume))
    x['unknown'] = faces.count('unknown')
    print(x)
    keys = [j for j in x.keys()]
    values = [j for j in x.values()]

    sns.set_style('dark')
    acc = x[name] * 100 / sum(values)

    sns.barplot(x=values, y=keys).set_title(f'{name.capitalize()} - acc: {acc}%')
    plt.show()

    print(x)
    exit(5)
elif method == '2':
    prompter = promptlib.Files()
    abs_path_to_video = prompter.file()
    should_be = input("Please enter person name:")
    file_name = abs_path_to_video.split('.')[0].split('/')[-1]
    output_faces = produce_video(abs_path_to_video, should_be)
    prompter.dst()

    print('Done.')


Writing frame 160 / 792 Accuracy=100.0%
Writing frame 170 / 792 Accuracy=100.0%
Writing frame 180 / 792 Accuracy=100.0%
Writing frame 190 / 792 Accuracy=100.0%
Writing frame 200 / 792 Accuracy=100.0%
Writing frame 210 / 792 Accuracy=100.0%
Writing frame 220 / 792 Accuracy=100.0%
Writing frame 230 / 792 Accuracy=100.0%
Writing frame 350 / 792 Accuracy=97.77777777777777%
Writing frame 440 / 792 Accuracy=97.95918367346938%
Writing frame 470 / 792 Accuracy=98.0%
Writing frame 510 / 792 Accuracy=98.14814814814815%
Writing frame 630 / 792 Accuracy=98.18181818181819%
Writing frame 640 / 792 Accuracy=96.61016949152543%
Done.


In [2]:
import os


def return_results(output_faces, should_be):
    for i in range(len(output_faces)):
        output_faces[i] = output_faces[i].lower()

    if len(output_faces) == 0:
        print('No faces found')
        output_faces = []

    acc = output_faces.count(should_be) * 100 / len(output_faces)
    print(acc)

    known_encodings = get_known_people_from_encodings()
    for i in range(len(known_encodings)):
        known_encodings[i] = known_encodings[i].lower()

    names = {}
    for name in known_encodings:
        names[name] = output_faces.count(name.lower())
    names['unknown'] = output_faces.count('unknown')
    return [file_name.lower(), names, acc]


list = []
if len(output_faces) == 0:
    list[1] = 'noface'
else:
    list = return_results(output_faces, should_be)
print(list)

if len(output_faces) == 0:
    list[1] = 'noface'
try:
    os.mkdir(f'results/{should_be}')
except OSError:
    print("Creation of the directory %s failed" % should_be)

with open(f'results/{should_be}/{list[0]}.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(list, filehandle)

96.72131147540983
['daniel', {'stefan': 4, 'yt4': 0, 'diana': 0, 'f': 0, 'tomsa_s': 0, 'costis': 0, 'raluca': 0, 'cosmin': 0, 'yt2': 0, 'costi': 0, 'alex': 0, 'yt3': 0, 'test_person_1': 0, 'twin2': 0, 'daniel': 118, 'twin1': 0, 'yt1': 0, 'mom': 0, 'ioana': 0, 'andra': 0, 'raed': 0, 'unknown': 0}, 96.72131147540983]


In [3]:

import pickle
import matplotlib.pyplot as plt
import os
import seaborn as sns


def plot_all_data():
    directory = 'results'
    objs = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and filename.endswith('data'):
            with open(f, 'rb') as filehandle:
                # read the data as binary data stream
                while 1:
                    try:
                        objs.append(pickle.load(filehandle))
                    except EOFError:
                        break
    sns.set_style('darkgrid')
    for results in objs:
        actual_name = results[0]
        output_faces = results[1]
        if output_faces == 'noface':
            sns.barplot(x=values, y=keys).set_title(actual_name + '- no face found')
            plt.show()
        else:
            acc = results[2]
            print('Accuracy= ', acc, '%')

            keys = [j for j in output_faces.keys()]
            values = [j for j in output_faces.values()]
            if actual_name == 'yt3(2)':
                sns.barplot(x=values, y=keys).set_title(actual_name + 'interchanged train with test')
            else:
                sns.barplot(x=values, y=keys).set_title(actual_name)

            plt.show()


plot_all_data()